# Just an exercize


In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms.functional as TF


In [ ]:
class DoubleConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(DoubleConv, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 3, 1, 1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, 3, 1, 1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.conv(x)


In [ ]:
class UNET(nn.Module):
    def __init__(self, in_channels=3, out_channels=1, features=[64, 128, 256, 512]):
        super(UNET, self).__init__()
        self.ups = nn.ModuleList()
        self.downs = nn.ModuleList()
        self.pool2d = nn.MaxPool2d(kernel_size=2, stride=2)

        for feature in features:
            self.downs.append(DoubleConv(in_channels, feature))
            in_channels = feature

        for feature in reversed(features):
            self.ups.append(nn.ConvTranspose2d(
                feature*2, feature, kernel_size=2, stride=2))
            self.ups.append(DoubleConv(feature*2, feature))

        self.bottleneck = DoubleConv(features[-1], features[-1]*2)
        self.finalConv = nn.Conv2d(features[0], out_channels, kernel_size=1)

    def forward(self, x):
        skip_conn = []
        for down in self.downs:
            x = down(x)
            skip_conn.append(x)
            x = self.pool2d(x)

        x = self.bottleneck(x)
        skip_conn = skip_conn[::-1]

        for idx in range(0, len(self.ups), 2):
            x = self.ups[idx](x)
            skip_co = skip_conn[idx//2]

            if x.shape != skip_co.shape:
                x = TF.resize(x, size=skip_co.shape[2:])

            concat_skip = torch.cat((skip_co, x), dim=1)
            x = self.ups[idx+1](concat_skip)

        return self.finalConv(x)


In [ ]:
x = torch.randn((3, 1, 16, 16))
model = UNET(in_channels=1, out_channels=1)
preds = model(x)
# print('prediction: {}'.format(preds))
# print("original: {}".format(x))

assert preds.shape == x.shape


In [ ]:
import os
from PIL import Image
from torch.utils.data import Dataset
import numpy as np
from dataset import KomatsDataset


class KomatsDataset(Dataset):
    def __init__(self, img_dir, mask_dir, transform=None):
        self.img_dir = img_dir
        self.mask_dir = mask_dir
        self.transform = transform
        self.imgs = os.listdir(img_dir)

    def __len__(self):
        return len(self.imgs)

    def __getitem__(self, index):
        img_path = os.path.join(self.img_dir, self.imgs[index])
        mask_path = os.path.join(self.mask_dir, self.imgs[index].replace("rgb", "label"))
        image = np.array(Image.open(img_path).convert("RGB"))
        mask = np.array(Image.open(mask_path).convert("L"), dtype=np.float32)
        mask[mask != 0] = 1.0

        if self.transform is not None:
            augmentation = self.transform(image=image, mask=mask)
            image = augmentation["image"]
            mask = augmentation["mask"]


        return image, mask

In [ ]:
import albumentations as A
from albumentations.pytorch import ToTensorV2
from tqdm import tqdm 
import torch.optim as optim 
from dataset import KomatsDataset
from utils import (load_checkpoint, save_checkpoint, get_loaders, check_accuracy, save_predictions_as_imgs)


# Hyper Parameters

In [ ]:
LEARNING_RATE = 1E-4
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
BATCH_SIZE = 3
NUM_EPOCHS = 10
NUM_WORKERS = 3
IMAGE_HEIGHT = 480
IMAGE_WIDTH = 480
PIN_MEMORY = True
LOAD_MODEL = True
TRAIN_IMG_DIR = "..\dataset\multi\multi_plant"
TRAIN_IMG_LABEL = "..\dataset\multi\multi_label"
VAL_IMG_DIR = "..\dataset\multi\multi_plant_val"
VAL_IMG_LABEL = "..\dataset\multi\multi_label_val"


In [ ]:
def train_fn(loader, model, optimizer, loss_fn, scaler):
    loop = tqdm(loader)
    for batch_idx, (data,  targets) in enumerate(loop):
        data = data.to(device=DEVICE)
        targets = targets.float().unsqueeze(1).to(device=DEVICE)

        with torch.cuda.amp.autocast():
            predictions = model(data)
            loss = loss_fn(predictions, targets)
             
        optimizer.zero_grad()
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        loop.set_postfix(loss=loss.item())



    

In [ ]:
train_transform = A.Compose(
    [
        A.Resize(height=IMAGE_HEIGHT, width=IMAGE_WIDTH),
        A.Rotate(limit=35, p=1.0),
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.5),
        A.Normalize(
            mean=[0.0,0.0,0.0],
            std=[1.0,1.0,1.0],
            max_pixel_value=255.0
        ),
        ToTensorV2()
    ]

)
val_transpose = A.Compose(
    [
        A.Resize(height=IMAGE_HEIGHT, width=IMAGE_WIDTH),
        A.Normalize(
            mean=[0.0, 0.0, 0.0],
            std=[1.0, 1.0, 1.0],
            max_pixel_value=255.0,
        ),
        ToTensorV2(),
    ]
)

model = UNET(in_channels=3, out_channels=1).to(DEVICE)
loss_fn = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [ ]:
train_loader, val_loader = get_loaders(
    TRAIN_IMG_DIR,
    TRAIN_IMG_LABEL,
    VAL_IMG_DIR,
    VAL_IMG_LABEL,
    BATCH_SIZE,
    train_transform,
    val_transpose,
    NUM_WORKERS,
    PIN_MEMORY
)

scaler = torch.cuda.amp.GradScaler()

for epoch in range(NUM_EPOCHS):
    train_fn(train_loader, model, optimizer, loss_fn, scaler)
    checkpoint = {
        "state_dict": model.state_dict(),
        "optimizer":optimizer.state_dict(),
    }
    save_checkpoint(checkpoint)

    # check accuracy
    check_accuracy(val_loader, model, device=DEVICE)

    # print some examples to a folder
    save_predictions_as_imgs(
        val_loader, model, folder=".\saved_images", device=DEVICE
    )

